In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split


from input_generator import load_raw_data
from data_parser import get_tags_and_labels, get_vocab, get_tags

In [20]:
csvpath = 'data/CAvideos.cvs'
tab_dataframe = get_tags_and_labels(csvpath)

tab_dataframe.dropna(inplace = True) 
   
tab_dataframe["tags"]= tab_dataframe["tags"].str.split(".", n = 1, exp) 
tab_dataframe

,tags,category_id
0,Eminem,10
1,plush,23
2,racist superman,23
3,ryan,24
4,edsheeran,10
5,#DramaAlert,25
6,Funny Moments,23
7,SHANtell martin,22
8,logan paul vlog,24
9,God,22
